# George D. Crochiere - 0961739
## EE361 - Project 2
## 03/25/2024

In [30]:
import locale
def getpreferredencoding(do_setlocale = True):
    return "UTF-8"

locale.getpreferredencoding = getpreferredencoding
!pip install chainer
import cupy
# Note: if you cannot connect to GPU, then import cupy will not work

# from google.colab import drive
# drive.mount('/content/drive')
# Allow connection to Google Drive to save and access images


[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


# Double Image

In [17]:
import cupy as cp
from PIL import Image

# CUDA kernel code
double_kernel = cp.RawKernel(r'''
extern "C" __global__
void doubleImg(const unsigned char* old_im, unsigned char* new_im, int width, int height)
{
    int tid_x = blockDim.x * blockIdx.x + threadIdx.x;
    int tid_y = blockDim.y * blockIdx.y + threadIdx.y;

    for (int i = 0; i < 3; i++) {
        for (int j = 0; j < 3; j++) {
            int x = tid_x * 3 + i;
            int y = tid_y * 3 + j;

            if (x < width && y < height) {
                int tid = y * width * 3 + x * 3;
                int tid2ds[] = [
                    tid,
                    (y + 1) * width * 3 + (x) * 3,
                    (y) * width * 3 + (x + 1) * 3,
                    (y + 1) * width * 3 + (x + 1) * 3
                ];

                for (int k = 0; k < 3; k++) {
                    unsigned char c = old_im[tid + k];

                    for (int tid2d : tid2ds) {
                        new_im[tid2d + k] = c;
                    }
                }
            }
        }
    }
}

''', 'doubleImg')

def makeDouble(imageFile):
    # Load image using PIL
    oldImage = Image.open(imageFile).convert('RGB')
    width, height = oldImage.size

    # Convert image to cupy array
    oldIm = cp.asarray(oldImage, dtype=cp.ubyte)
    newIm = cp.empty_like(oldIm)

    # Reshape arrays for easier parallelization
    size = width * height
    oldIm = cp.reshape(oldIm, (size, 3),    newIm = cp.reshape(newIm, (size * * 2,))

    # Launch CUDA kernel
    block_dim = (32, 32)
    grid_dim = (int (cp.ceil(width/3/block_dim[0])), int (cp.ceil(height/3/block_dim[1])))

    t3 = time.perf_counter()
    double_kernel((grid_dim), (block_dim), (oldIm, newIm, width, height))
    t4 = time.perf_counter()
    print('time taken to run:',t4-t3)

    # Reshape newIm array back to image dimensions
    newIm = cp.reshape(newIm, (he * 2ight, w * 2idth, 3))

    # Convert cupy array to PIL Image
    newImage = Image.fromarray(cp.asnumpy(newIm))

    # Save the image
    newImage.save('./Dole1D.jpg')

import time
if __name__ == '__main__':
    t1 = time.perf_counter()
    makeDouble('./blue-forest-minimal-4k-kz.jpg')
    t2 = time.perf_counter()
    print('time taken to run:',t2-t1)


SyntaxError: invalid syntax (4086832430.py, line 51)

# Vertical Flip

In [39]:
import cupy
import time
from PIL import Image

# CUDA Kernel - Vertical Flip
verticalFlip_kernel = cp.RawKernel(r'''
extern "C" __global__
void verticalFlip(const unsigned char* old_im, unsigned char* new_im, int width, int height) {
    int tid_x = blockDim.x * blockIdx.x + threadIdx.x;
    int tid_y = blockDim.y * blockIdx.y + threadIdx.y;
    
    int x = tid_x * 3;
    int y = tid_y * 3;

    if (x < width && y < height) {
        int tid = y * width * 3 + x * 3;

        unsigned char r = old_im[tid];
        unsigned char g = old_im[tid + 1];
        unsigned char b = old_im[tid + 2];
        
        int tid_new = (y * width * 3) + ((width - x) * 3);
        
        new_im[tid_new] = r;
        new_im[tid_new + 1] = g;
        new_im[tid_new + 2] = b;
    }
}
''', 'verticalFlip')

def makeVerticalFlip(imgFile):
    # Original Image
    oldImage = Image.open(imgFile).convert('RGB')
    width, height = oldImage.size
    size = width * height
    
    # Get image arrays and reshape
    oldImArr = cp.asarray(oldImage, dtype=cp.ubyte)
    newImArr = cp.empty_like(oldImArr)
    oldImArr = cp.reshape(oldImArr, (size, 3))
    newImArr = cp.reshape(newImArr, (size, 3))
    
    # CUDA config
    block_dim = (32,32)
    grid_dim = (int (cp.ceil(width / 3 / block_dim[0])), int(cp.ceil(height / 3 / block_dim[1])))
    
    # Start and run
    t1Vertical = time.perf_counter()
    verticalFlip_kernel((grid_dim), (block_dim), (oldImArr, newImArr, width, height))
    t2Vertical = time.perf_counter()
    print('time taken to run kernel:', t2Vertical - t1Vertical)
    
    # Reshape and convert
    newImArr = cp.reshape(newImArr, (height, width, 3))
    newImage = Image.fromarray(cp.asnumpy(newImArr))
    newImage.save('./VerticalFlipResult.jpg')
    
# Main Method
if __name__ == '__main__':
    tVerticalStart = time.perf_counter()
    makeVerticalFlip('./landscape-scenery-minimalist-4k-r6-2560x1440.jpg');
    tVerticalStop = time.perf_counter()
    print('time taken to run:', tVerticalStop - tVerticalStart)
    

time taken to run kernel: 0.043225499999607564
time taken to run: 0.11289839999972173


This operation works by taking the index location for each entry within the old image data array, converting it to the appropiate (x, y) coordinate location and then finding the appropiate coordinate if the pixel were to be flipped along the vertical axis. This new coordinate was then converted to an index associated on the new image array, which then has the associated RGB values place.

This process is split up amongst the different threads on each of the various GPGPU CUDA Cores, allowing each pixel to be calculated in parallel and to be combined afterwards.

Original:
![Original Image](./landscape-scenery-minimalist-4k-r6-2560x1440.jpg)

Flipped across the Vertical Axis:
![Flipped Image](./VerticalFlipResult.jpg)

# Edge Detection

In [40]:
import cupy as cp
from PIL import Image

# CUDA kernel code
edge_kernel = cp.RawKernel(r'''
extern "C" __global__
void edgeDetection(const unsigned char* old_im, unsigned char* new_im, int size)
{
    int tid_x = blockDim.x * blockIdx.x + threadIdx.x;
    int tid_y = blockDim.y * blockIdx.y + threadIdx.y;
    
    int x = tid_x * 3;
    int y = tid_y * 3;
    
    int xArr[][] = {{-1, 0, 1}, {-2, 0, 2}, {-1, 0, 1}};
    int yArr[][] = {{1, 2, 1}, {0, 0, 0}, {-1, -2, -1}};
    
    if ((x < width - 1) && (x > 0) && (y < height - 1) && (y > 0)) {
        int xScore = 0;
        int yScore = 0;
        
        for (int xVar = -1; xVar <= 1; xVar++) {
            for (int yVar = -1; yVar <= 1; yVar++) {
                unsigned char r = ((y + yVar) * width * 3 + (x + xVar) * 3)
                unsigned char g = ((y + yVar) * width * 3 + (x + xVar) * 3) + 1
                unsigned char b = ((y + yVar) * width * 3 + (x + xVar) * 3) + 2
                
                unsigned char gray = (unsigned char)((r + g + b) / 3);
                
                xScore += (xArr[xVar + 1] * gray);
                yScore += (yArr[yVar + 1] * gray);
            }
        }
        
        int gScore = sqrt((xScore * xScore) + (yScore * yScore));
        int tid = y * width * 3 + x * 3;
        
        for (i = 0; i < 3; i++) {
            new_im[tid + i] = (gScore > 175) * 255;
        }
        
    } else if ((x < width) && (y < height)) {
        int tid = y * width * 3 + x * 3;
        for (i = 0; i < 3; i++) {
            new_im[tid + i] = 0;
        }
    }
}

''', 'edgeDetection')

def makeEdgeDetection(imgFile):
    # Original Image
    oldImage = Image.open(imgFile).convert('RGB')
    width, height = oldImage.size
    size = width * height
    
    # Get image arrays and reshape
    oldImArr = cp.asarray(oldImage, dtype=cp.ubyte)
    newImArr = cp.empty_like(oldImArr)
    oldImArr = cp.reshape(oldImArr, (size, 3))
    newImArr = cp.reshape(newImArr, (size, 3))
    
    # CUDA config
    block_dim = (32,32)
    grid_dim = (int (cp.ceil(width / 3 / block_dim[0])), int(cp.ceil(height / 3 / block_dim[1])))
    
    # Start and run
    t1Vertical = time.perf_counter()
    edge_kernel((grid_dim), (block_dim), (oldImArr, newImArr, width, height))
    t2Vertical = time.perf_counter()
    print('time taken to run kernel:', t2Vertical - t1Vertical)
    
    # Reshape and convert
    newImArr = cp.reshape(newImArr, (height, width, 3))
    newImage = Image.fromarray(cp.asnumpy(newImArr))
    newImage.save('./VerticalFlipResult.jpg')
    
# Main Method
if __name__ == '__main__':
    tVerticalStart = time.perf_counter()
    makeEdgeDetection('./landscape-scenery-minimalist-4k-r6-2560x1440.jpg');
    tVerticalStop = time.perf_counter()
    print('time taken to run:', tVerticalStop - tVerticalStart)

CompileException: C:\Users\Georg\AppData\Local\Temp\tmpg2qdhbv4\b24bd39836267584502ae66a2ce52956ca5b7b2c.cubin.cu(11): error: an array may not have elements of this type
      int xArr[][] = {{-1, 0, 1}, {-2, 0, 2}, {-1, 0, 1}};
                ^

C:\Users\Georg\AppData\Local\Temp\tmpg2qdhbv4\b24bd39836267584502ae66a2ce52956ca5b7b2c.cubin.cu(12): error: an array may not have elements of this type
      int yArr[][] = {{1, 2, 1}, {0, 0, 0}, {-1, -2, -1}};
                ^

C:\Users\Georg\AppData\Local\Temp\tmpg2qdhbv4\b24bd39836267584502ae66a2ce52956ca5b7b2c.cubin.cu(14): error: identifier "width" is undefined
      if ((x < width - 1) && (x > 0) && (y < height - 1) && (y > 0)) {
               ^

C:\Users\Georg\AppData\Local\Temp\tmpg2qdhbv4\b24bd39836267584502ae66a2ce52956ca5b7b2c.cubin.cu(14): error: identifier "height" is undefined
      if ((x < width - 1) && (x > 0) && (y < height - 1) && (y > 0)) {
                                             ^

C:\Users\Georg\AppData\Local\Temp